In [1]:
import os
import pandas as pd

In [2]:
files_top3 = [i for i in os.listdir() if i.startswith('top_3')]


In [3]:
def clean_clm_name(dframe):
    return [i.lower().strip().replace(' ','_') for i in dframe.columns]

def clean_clm_name_dframe(dframe):
    dframe.columns = [i.lower().strip().replace(' ','_') for i in dframe.columns]
    return dframe

In [4]:

top3_df = pd.DataFrame()
for i in files_top3:
    df = clean_clm_name_dframe(pd.read_csv(i))
    df['region'] = i.split('_')[-1][:-4]
    top3_df = top3_df.append(df)

C:\Users\nprol195\AppData\Local\Temp\ipykernel_22472\1642513292.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top3_df = top3_df.append(df)
C:\Users\nprol195\AppData\Local\Temp\ipykernel_22472\1642513292.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top3_df = top3_df.append(df)


In [5]:
store_df = pd.read_csv('store_lookup.csv')
store_df.columns = clean_clm_name(store_df)
store_top3_df = store_df.merge(top3_df,how='inner',left_on='storeid',right_on='store').drop(columns=['store', 'region_y']).reset_index(drop=True).copy()
store_top3_df.rename(columns={'region_x':'region'},inplace=True)

In [6]:
all_df = pd.read_csv('all_data.csv')
all_df = clean_clm_name_dframe(all_df)
all_df = all_df.drop(columns=['order_id']).groupby(['store_name','region']).sum().reset_index().copy()

In [7]:
store_top3_df['store_name'] = store_top3_df['store_name'].str.lower().str.strip()
all_df['store_name'] = all_df['store_name'].str.lower().str.strip()

In [8]:
final_df = store_top3_df.merge(all_df,how='left',on='store_name').loc[:,['store_name', 'region_x', 'sales_person',
       'percent_of_store_sales', 'sale_value']].copy()


In [9]:
final_df['sales_per_associate'] = (final_df['percent_of_store_sales']/100) * final_df['sale_value']
final_df.rename(columns={'region_x':'region','sale_value':'total_store_sales'},inplace=True)
final_df

,store_name,region,sales_person,percent_of_store_sales,total_store_sales,sales_per_associate
0,lewisham,East,ML,26,114420.21,29749.2546
1,lewisham,East,PR,21,114420.21,24028.2441
2,lewisham,East,JK,17,114420.21,19451.4357
3,wimbledon,West,YP,14,107841.68,15097.8352
4,wimbledon,West,FS,13,107841.68,14019.4184
5,wimbledon,West,FH,10,107841.68,10784.1680
6,dulwich,East,CA,21,106613.79,22388.8959
7,dulwich,East,JM,15,106613.79,15992.0685
8,dulwich,East,TP,12,106613.79,12793.6548
9,chelsea,West,JS,25,88886.31,22221.5775


In [10]:
final_df_other = final_df[['store_name','region','percent_of_store_sales','sales_per_associate']].groupby(['store_name','region']).sum().reset_index().copy()

In [11]:
final_df_other['percent_of_store_sales'] = 100 - final_df_other['percent_of_store_sales']
final_df_other['sales_per_associate'] = all_df['sale_value'] - final_df_other['sales_per_associate']
final_df_other['sales_person'] = 'OTH'

final_df_other['total_store_sales'] = all_df['sale_value']
final_df_other = final_df_other[['store_name', 'region', 'sales_person', 'percent_of_store_sales',
                     'total_store_sales', 'sales_per_associate']]

# [['store_name', 'region', 'sales_person', 'percent_of_store_sales',
#        'sales_per_associate']]

In [12]:
final_df_other.append(final_df).sort_values(['region','store_name','percent_of_store_sales']).reset_index(drop=True).to_csv('final.csv',index=False)
os.startfile('final.csv')

C:\Users\nprol195\AppData\Local\Temp\ipykernel_22472\462519948.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df_other.append(final_df).sort_values(['region','store_name','percent_of_store_sales']).reset_index(drop=True).to_csv('final.csv',index=False)
